<a href="https://www.kaggle.com/code/bedirhankaraahmetli/vacation-preferences-between-mountains-and-beaches?scriptVersionId=217849087" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **0 - Introduction**

## **Project Overview**

In recent years, understanding consumer preferences has become a cornerstone of personalized marketing and service delivery. One area where such understanding is particularly valuable is in the tourism and travel industry. People’s vacation choices, such as preferring mountains or beaches, are influenced by various demographic and lifestyle factors, including age, income, travel habits, and environmental concerns. Analyzing these factors and predicting preferences can help businesses optimize their offerings and provide a more tailored customer experience.

This project focuses on predicting vacation preferences between mountains and beaches using demographic data. By leveraging a dataset of over 52,000 individuals, we aim to explore patterns and relationships in the data, apply data mining techniques to extract insights, and develop machine learning models to forecast preferences. The project will integrate various stages, including data exploration, preprocessing, model building, evaluation, deployment, and results interpretation.

## **Objective**

The primary objective of this project is to build predictive models that classify individuals' vacation preferences as either "mountains" or "beaches" based on their demographic and lifestyle attributes. Additionally, the project seeks to achieve the following goals:

- Data Analysis: Explore and understand the dataset to uncover trends and relationships that influence vacation preferences.
- Model Development: Experiment with various machine learning algorithms to identify the most effective approach for prediction.
- Feature Insights: Determine the importance of specific features (e.g., income, proximity to vacation spots) in predicting preferences.
- Practical Application: Deploy a predictive model that can be used by travel-related businesses to personalize marketing campaigns and services.
- Educational Value: Gain hands-on experience in applying data mining and machine learning techniques in a real-world scenario.

## **Dataset Overview**

The dataset used for this project comprises 52,444 entries and 13 features, offering a comprehensive view of the factors that may influence vacation preferences. Each entry represents an individual, with attributes categorized as numerical, categorical, or binary. Below is a summary of the dataset features:

- Age: Age of the individual (numerical).
- Gender: Gender identity (categorical: male, female, non-binary).
- Income: Annual income of the individual (numerical).
- Education Level: Highest level of education attained (categorical: high school, bachelor, master, doctorate).
- Travel Frequency: Number of vacations taken per year (numerical).
- Preferred Activities: Activities preferred during vacations (categorical: hiking, swimming, skiing, sunbathing).
- Vacation Budget: Budget allocated for vacations (numerical).
- Location: Type of residence (categorical: urban, suburban, rural).
- Proximity to Mountains: Distance from the nearest mountains (numerical, in miles).
- Proximity to Beaches: Distance from the nearest beaches (numerical, in miles).
- Favorite Season: Preferred season for vacations (categorical: summer, winter, spring, fall).
- Pets: Ownership of pets (binary: 0 = No, 1 = Yes).
- Environmental Concerns: Environmental awareness or concerns (binary: 0 = No, 1 = Yes).

This diverse dataset allows for the application of multiple data mining and machine learning techniques, making it ideal for predictive modeling. Through careful analysis and modeling, this project aims to provide meaningful insights into vacation preferences and build an effective predictive tool.

# **1 - Data Understanding and Preprocessing**

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1.1 Loading the Dataset

In [ ]:
filepath = "/kaggle/input/mountains-vs-beaches-preference/mountains_vs_beaches_preferences.csv"
df = pd.read_csv(filepath, comment='#')

## 1.2 Inspecting dataset

In [ ]:
def inspect_dataset(dataframe):
    print("\nDataset Info:")
    dataframe.info()
    print("\nSummary Statistics:")
    print(dataframe.describe())
    print("\nChecking for missing values:")
    print(dataframe.isnull().sum())

inspect_dataset(df)

In [ ]:
# Splitting features
numerical_features = ['Age', 'Income', 'Travel_Frequency', 'Vacation_Budget', 'Proximity_to_Mountains', 'Proximity_to_Beaches']
categorical_features = ['Gender', 'Education_Level', 'Preferred_Activities', 'Location', 'Favorite_Season']
binary_features = ['Pets', 'Environmental_Concerns']

In [ ]:
print("\nChecking for missing values:")
print(df.isnull().sum())

We can clearly say there isn't any missing values. Therefore, no further steps are necessary about handling missing values

## 1.3 Detecting and Handling Outliers

In [ ]:
def plot_outliers(dataframe, numerical_columns):
    plt.figure(figsize=(16, 8))
    for i, col in enumerate(numerical_columns, start=1):
        plt.subplot(2, 3, i)
        sns.boxplot(y=dataframe[col])
        plt.title(f"Boxplot of {col}")
        plt.ylabel('')
        plt.xlabel(col)
    plt.tight_layout()
    plt.show()

# Visualize outliers for numerical features
plot_outliers(df, numerical_features)

# Numerical validation of outliers using IQR
def detect_outliers(dataframe, column):
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    outliers = dataframe[(dataframe[column] < Q1 - 1.5 * IQR) | (dataframe[column] > Q3 + 1.5 * IQR)]
    return outliers

# Check for outliers in numerical features
for col in numerical_features:
    outliers = detect_outliers(df, col)
    print(f"{col}: {len(outliers)} outliers detected.")

We can see from the boxplots above,
there aren't any outliers detected for numerical fetaures

## 1.4 Encoding Categorical Features

In [ ]:
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

In [ ]:
print('\nDataset After Encoding:')
df.head()

## 1.5 Normalizing Numerical Features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [ ]:
print('\nDataset After Scaling:')
df.head()

In [ ]:
# Boxplots for numerical features after encoding and scaling
plt.figure(figsize=(16, 8))
for i, col in enumerate(numerical_features, start=1):
    plt.subplot(2, 3, i)
    sns.boxplot(y=df[col])
    plt.title(f"Boxplot of {col} After Scaling")
    plt.ylabel('')
    plt.xlabel(col)
plt.tight_layout()
plt.show()

In [ ]:
# Histograms for numerical features after encoding and scaling
plt.figure(figsize=(16, 8))
for i, col in enumerate(numerical_features, start=1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], kde=True, bins=20)
    plt.title(f"Histogram of {col} After Scaling")
    plt.xlabel(col)
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# 2 - Data Preparation

In [ ]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split, StratifiedKFold

Since there isn't much feature, we decided to do feature selection out of the k-fold cross validation loop once, then proceed to the model training steps

## 2.1 Feature Selection

### 2.1.1 Correlation Matrix

In [ ]:
# Correlation Analysis
def plot_correlation_matrix(dataframe):
    correlation_matrix = dataframe.corr()
    plt.figure(figsize=(16, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Correlation Matrix')
    plt.show()
    return correlation_matrix

correlation_matrix = plot_correlation_matrix(df)

In [ ]:
# Creating a list to store features with correlation greater than the threshold
correlated_features = set()
threshold = 0.8

In [ ]:
# Identifying highly correlated features
correlated_features = set()
threshold = 0.8

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            feature_1 = correlation_matrix.columns[i]
            feature_2 = correlation_matrix.columns[j]
            correlated_features.add((feature_1, feature_2))

# Print pairs of highly correlated features
print(f"Highly Correlated Feature Pairs (threshold > {threshold}): {correlated_features}")


Based on the correlation matrix there isn't features highly correlated (> 0.8) with each other

### 2.1.2 Recursive Feature Elimination (RFE)

In [ ]:
# Importing necessary libraries
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Define features (X) and target (y)
X = df.drop(columns='Preference')
y = df['Preference']

In [ ]:
# RFE using RandomForestClassifier
rfe_model = RandomForestClassifier(random_state=1905)
rfe = RFE(estimator=rfe_model, n_features_to_select=10)
X_rfe = rfe.fit_transform(X, y)

In [ ]:
# Extracting selected feature names
selected_features = X.columns[rfe.support_]
print(f"Selected Features: {selected_features}")

In [ ]:
# Filter the dataset with selected features
X = df[selected_features]
y = df['Preference']

In [ ]:
# Verify the filtered dataset
print("\nFiltered Dataset with Selected Features:")
print(X.head())

# **3 - Model Training and Evaluation**

In [ ]:
# Importing necessary libraries
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
)
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import itertools

In [ ]:
# Suppressing unnecessary warnings to keep the output clean
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Splitting the dataset into train and test sets
print("Splitting the dataset into training and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size=0.1, random_state=1905, stratify=y)
print(f"Training set size: {X_train.shape}, Test set size: {X_test.shape}")

In [ ]:
# Initializing 9-fold cross-validation
print("\nInitializing 9-fold cross-validation...")
n_splits = 9
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1905)
folds = list(skf.split(X_train, y_train))
print(f"Number of folds: {n_splits}")

In [ ]:
# Defining models and their hyperparameter grids
print("\nDefining models and their hyperparameter grids...")
models = {
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=1000, random_state=1905, solver='liblinear'),
        "param_grid": {
            "penalty": ["l1", "l2"],
            "C": [0.01, 0.1, 1, 10]
        }
    },
    "Decision Tree": {
        "model": DecisionTreeClassifier(random_state=1905),
        "param_grid": {
            "max_depth": [None, 5, 7, 10],
            "min_samples_split": [2, 5, 10]
        }
    },
    "Random Forest": {
        "model": RandomForestClassifier(random_state=1905),
        "param_grid": {
            "n_estimators": [50, 75, 100],
            "max_depth": [None, 5, 10],
            "min_samples_split": [2, 5, 10]
        }
    },
    "Support Vector Machine": {
        "model": SVC(random_state=1905, probability=True), # `probability=True` enables probability predictions
        "param_grid": {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf"]
        }
    },
    "Naive Bayes": {
        "model": GaussianNB(),
        "param_grid": {}  # No hyperparameters to tune
    },
    "K-Nearest Neighbors": {
        "model": KNeighborsClassifier(),
        "param_grid": {
            "n_neighbors": [3, 5, 7, 9],
            "weights": ["uniform", "distance"]
        }
    }
}
print("Models and hyperparameter grids defined successfully.")

In [ ]:
# Grid search function
def custom_grid_search(model, X_train, y_train, X_val, y_val, param_grid):
    """
    Perform grid search for hyperparameter tuning.
    If the parameter grid is empty, train the model directly.
    """
    print("\tPerforming grid search...")
    best_params = None  # To store the best parameter combination
    best_score = 0  # To store the highest validation accuracy
    results = []  # To store all parameter combinations and their corresponding scores

    # If no parameters to tune, train the model directly
    if not param_grid:
        print("\tNo parameters to tune for this model. Training directly...")
        model.fit(X_train, y_train)  # Train the model
        val_accuracy = model.score(X_val, y_val)  # Evaluate the model
        results.append(({}, val_accuracy))
        return {}, val_accuracy, results

    # Generate all combinations of parameters
    keys, values = zip(*param_grid.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Evaluate each parameter combination
    for params in param_combinations:
        try:
            # Set parameters and fit the model
            model.set_params(**params)
            model.fit(X_train, y_train)

            # Evaluate on validation set
            val_accuracy = model.score(X_val, y_val)
            results.append((params, val_accuracy))

            # Update best parameters if current accuracy is higher
            if val_accuracy > best_score:
                best_score = val_accuracy
                best_params = params
        except Exception as e:
            print(f"\tSkipping parameters {params} due to error: {e}")

    print(f"\tGrid search completed. Best parameters: {best_params}, Best score: {best_score:.4f}")
    return best_params, best_score, results

In [ ]:
# Function to plot grid search results
def plot_grid_search_results(results, fold_idx):
    """
    Plot grid search results for a specific fold using a horizontal bar chart.
    """
    print(f"\tPlotting grid search results for Fold {fold_idx + 1}...")
    results_df = pd.DataFrame([
        {"Params": str(params), "Accuracy": accuracy}
        for params, accuracy in results
    ])
    results_df.sort_values(by="Accuracy", ascending=False, inplace=True)

    plt.figure(figsize=(12, 8))
    sns.barplot(
        y=results_df["Params"],
        x=results_df["Accuracy"],
        palette="coolwarm",
        orient="h"
    )

    for i, acc in enumerate(results_df["Accuracy"]):
        plt.text(acc + 0.002, i, f"{acc:.4f}", va="center", ha="left", fontsize=10)

    plt.xlim(0.8, 1.0)  # Adjust the x-axis range for better visualization
    plt.xlabel("Accuracy")
    plt.ylabel("Parameter Combination")
    plt.title(f"Grid Search Results for Fold {fold_idx + 1}")
    plt.tight_layout()
    plt.show()

In [ ]:
# Metrics collection and evaluation
print("\nStarting model evaluation and metrics collection...")
all_results = []  # To store evaluation results for all models

In [ ]:
# Loop through each model
for model_name, model_info in models.items():
    print(f"\nStarting Cross-Validation for {model_name}...\n")
    model = model_info["model"]  # Extract model
    param_grid = model_info["param_grid"]  # Extract parameter grid
    
    # Cross-validation for each fold
    best_params_overall = None  # Best parameters across all folds
    best_score_overall = 0  # Best score across all folds
    fold_accuracies = []  # Store fold accuracies
    training_errors = []  # Store training errors
    validation_errors = []  # Store validation errors

    for fold_idx, (train_idx, val_idx) in enumerate(folds):
        print(f"\n\tStarting Fold {fold_idx + 1}...")
        
        # Split training and validation data for the current fold
        X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Perform grid search
        best_params, best_score, results = custom_grid_search(model, X_fold_train, y_fold_train, X_fold_val, y_fold_val, param_grid)
        fold_accuracies.append(best_score)  # Append fold score

        # Track the best overall parameters
        if best_score > best_score_overall:
            best_score_overall = best_score
            best_params_overall = best_params

        # Compute training and validation errors
        model.set_params(**best_params)
        model.fit(X_fold_train, y_fold_train)  # Train the model
        training_error = 1 - accuracy_score(y_fold_train, model.predict(X_fold_train))  # Training error
        validation_predictions = model.predict(X_fold_val)  # Validation predictions
        validation_error = 1 - accuracy_score(y_fold_val, validation_predictions)  # Validation error
        training_errors.append(training_error)
        validation_errors.append(validation_error)

        # Plot grid search results (skip for Naive Bayes)
        if model_name != "Naive Bayes":
            plot_grid_search_results(results, fold_idx)

        print(f"\tEnding Fold {fold_idx + 1}.")

    # Retrain the model with the best parameters on the full training set
    print(f"\nRetraining the {model_name} on the full training set using the best parameters...")
    model.set_params(**best_params_overall)
    model.fit(X_train, y_train)

    # Evaluate on the test set
    print(f"\nEvaluating {model_name} on the test set...")
    y_test_pred = model.predict(X_test)
    y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    # Compute test set metrics
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_report = classification_report(y_test, y_test_pred, output_dict=True)

    # Save test set metrics for model comparison
    all_results.append({
        "Model": model_name,
        "Best Params Overall": best_params_overall,
        "Average Fold Accuracy": np.mean(fold_accuracies),
        "Test Set Metrics": {
            "Accuracy": test_accuracy,
            "Precision": test_report["weighted avg"]["precision"],
            "Recall": test_report["weighted avg"]["recall"],
            "F1-Score": test_report["weighted avg"]["f1-score"]
        }
    })

    print(f"\nBest Parameters for {model_name}: {best_params_overall}")
    print(f"Average Accuracy across all folds for {model_name}: {np.mean(fold_accuracies):.4f}")
    print(f"Test Set Accuracy: {test_accuracy:.4f}\n")

    # Plot learning curves
    print(f"\nPlotting learning curves for {model_name}...")
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(training_errors) + 1), training_errors, label="Training Error", marker="o")
    plt.plot(range(1, len(validation_errors) + 1), validation_errors, label="Validation Error", marker="o")
    plt.title(f"Learning Curves for {model_name}")
    plt.xlabel("Fold")
    plt.ylabel("Error")
    plt.legend()
    plt.grid(alpha=0.5)
    plt.tight_layout()
    plt.show()

    # Plot ROC-AUC curve (if applicable)
    if y_test_prob is not None:
        print(f"Plotting ROC-AUC curve for {model_name}...")
        fpr, tpr, _ = roc_curve(y_test, y_test_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f"ROC-AUC (AUC = {roc_auc:.4f})")
        plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
        plt.title(f"ROC-AUC Curve for {model_name}")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend()
        plt.grid(alpha=0.5)
        plt.tight_layout()
        plt.show()

    # Plot confusion matrix
    print(f"Plotting confusion matrix for {model_name} on the test set...")
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=model.classes_)
    disp.plot(cmap="Blues")
    plt.title(f"Confusion Matrix for {model_name} on Test Set")
    plt.tight_layout()
    plt.show()

In [ ]:
# Compare models
print("\nCreating model comparison table...")
comparison_df = pd.DataFrame([
    {
        "Model": result["Model"],
        "Test Accuracy": result["Test Set Metrics"]["Accuracy"],
        "Test Precision": result["Test Set Metrics"]["Precision"],
        "Test Recall": result["Test Set Metrics"]["Recall"],
        "Test F1-Score": result["Test Set Metrics"]["F1-Score"]
    }
    for result in all_results
])
print(comparison_df)

In [ ]:
# Choose the best model
best_model_row = comparison_df.sort_values(by="Test Accuracy", ascending=False).iloc[0]
best_model_name = best_model_row["Model"]
print(f"\nBest Model: {best_model_name} based on Test Accuracy.")

In [ ]:
# Retrain the best model on the entire dataset
print(f"\nTraining the best model ({best_model_name}) on the entire dataset (including test set)...")
full_X = np.vstack((X_train, X_test))
full_y = pd.concat([y_train, y_test])
best_model = models[best_model_name]["model"]
best_model.set_params(**all_results[comparison_df[comparison_df["Model"] == best_model_name].index[0]]["Best Params Overall"])
best_model.fit(full_X, full_y)

In [ ]:
# Generate and predict 10 random instances
print("\nGenerating 10 random instances and predicting their outcomes...")
random_instances = np.random.rand(10, full_X.shape[1])
random_predictions = best_model.predict(random_instances)
print("\nRandom Instances Predictions:")
for i, prediction in enumerate(random_predictions, 1):
    print(f"Instance {i}: Predicted Class = {prediction}")